In [ ]:
# Sales Forecasting using ARIMA
# Author: Devendra Yadav

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Step 1: Data Preparation
np.random.seed(42)
date_range = pd.date_range(start='2015-01-01', end='2022-12-01', freq='MS')
trend = np.linspace(1000, 5000, len(date_range))
seasonality = 300 * np.sin(2 * np.pi * date_range.month / 12)
noise = np.random.normal(0, 200, len(date_range))
sales = trend + seasonality + noise
sales = np.where(sales < 0, 0, sales)
marketing_spend = np.random.randint(2000, 10000, len(date_range))
economic_index = np.random.normal(100, 5, len(date_range))
df = pd.DataFrame({
    'Date': date_range,
    'Sales': sales.round(2),
    'Marketing_Spend': marketing_spend,
    'Economic_Index': economic_index.round(2)
})
df.set_index('Date', inplace=True)

# Step 2: Exploratory Data Analysis
plt.figure(figsize=(14,6))
sns.lineplot(data=df, x='Date', y='Sales')
plt.title('Monthly Sales Over Time')
plt.grid(True)
plt.show()

# Decomposition
decomposition = seasonal_decompose(df['Sales'], model='additive', period=12)
decomposition.plot().set_size_inches(14, 10)
plt.show()

# Step 3: Stationarity Check and Transformation
adf_result = adfuller(df['Sales'])
df['Log_Sales'] = np.log(df['Sales'].replace(0, np.nan)).fillna(method='bfill')
df['Sales_diff'] = df['Log_Sales'] - df['Log_Sales'].shift(1)
df.dropna(inplace=True)
adf_diff_result = adfuller(df['Sales_diff'])

# Step 4: Train-Test Split
train = df.loc[:'2021-12-01']
test = df.loc['2022-01-01':]
exog_train = train[['Marketing_Spend', 'Economic_Index']]
exog_test = test[['Marketing_Spend', 'Economic_Index']]

# Step 5: Model Training (ARIMA + Exogenous)
model = SARIMAX(train['Log_Sales'], exog=exog_train, order=(1,1,1), seasonal_order=(1,1,1,12), enforce_stationarity=False, enforce_invertibility=False)
result = model.fit(disp=False)

# Step 6: Forecasting
forecast_log = result.predict(start=len(train), end=len(train)+len(test)-1, exog=exog_test)
forecast = np.exp(forecast_log)
comparison = pd.DataFrame({
    'Actual_Sales': np.exp(test['Log_Sales']),
    'Forecasted_Sales': forecast
})

# Step 7: Visualization
plt.figure(figsize=(14,6))
plt.plot(comparison.index, comparison['Actual_Sales'], label='Actual Sales', marker='o')
plt.plot(comparison.index, comparison['Forecasted_Sales'], label='Forecasted Sales', marker='o', linestyle='--')
plt.title('Actual vs Forecasted Sales (2022)')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)
plt.show()

# Step 8: Model Evaluation
mape = np.mean(np.abs((comparison['Actual_Sales'] - comparison['Forecasted_Sales']) / comparison['Actual_Sales'])) * 100
rmse = np.sqrt(mean_squared_error(comparison['Actual_Sales'], comparison['Forecasted_Sales']))
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

# Conclusion
# The model demonstrated strong predictive performance and can be used to support strategic sales planning.
